In [1]:
from pathlib import Path, PurePath
import numpy as np, imageio.v3 as iio, pickle, torch
import pysaliency

stim   = pysaliency.read_json('MIT1003_twosize/stimuli.json')
fix    = pickle.load(open('MIT1003_twosize/scanpaths_twosize.pkl','rb'))
w,h    = 1024,768

# 1. out-of-bounds rate
xs,ys  = fix.train_xs.flatten(), fix.train_ys.flatten()
oob    = ((xs<0)|(xs>=w)|(ys<0)|(ys>=h)).mean()
print('out-of-bounds fixes %:', oob*100)

# 2. grid stride check
downsample, readout = 2,14
print('effective stride px:', downsample*readout)

# 3. center-bias sanity
cb = CrossvalidatedBaselineModel(stim, fix, bandwidth=10**-1.666767, eps=10**-14.88419)
ll = cb.information_gain(stim, fix, average='image')
print('baseline IG (should be ~0):', ll)


AttributeError: module 'pysaliency' has no attribute 'read_json'